# Módulo 5 Tarefa 1
## Base de nascidos vivos do DataSUS
O DataSUS disponibiliza diversos arquivos de dados com relação a seus segurados, conforme a [lei da transparência de informações públicas](https://www.sisgov.com/transparencia-acesso-informacao/#:~:text=A%20Lei%20da%20Transpar%C3%AAncia%20(LC,em%20um%20site%20na%20internet.).

Essas informações podem ser obtidas pela internet [aqui](http://www2.datasus.gov.br/DATASUS/index.php?area=0901&item=1). Como o processo de obtenção desses arquivos foge um pouco do nosso escopo, deixamos o arquivo ```SINASC_RO_2019.csv``` já como vai ser encontrado no DataSUS. O dicionário de dados está no arquivo ```estrutura_sinasc_para_CD.pdf``` (o nome do arquivo tal qual no portal do DataSUS).

### Nosso objetivo
Queremos deixar uma base organizada para podermos estudar a relação entre partos com risco para o bebê e algumas condições como tempo de parto, consultas de pré-natal etc.

#### Preparação da base
1. Carregue a base 'SINASC_RO_2019.csv'. Conte o número de registros e o número de registros não duplicados da base. Dica: você aprendeu um método que remove duplicados, encadeie este método com um outro método que conta o número de linhas. **Há linhas duplicadas?**  

2. Conte o número de valores *missing* por variável.  

3. Ok, no item anterior você deve ter achado pouco prático ler a informação de tantas variáveis, muitas delas nem devem ser interesantes. Então crie uma seleção dessa base somente com as colunas que interessam. São elas:
```
['LOCNASC', 'IDADEMAE', 'ESTCIVMAE', 'ESCMAE', 'QTDFILVIVO',
    'GESTACAO', 'GRAVIDEZ', 'CONSULTAS', 'APGAR5']
```
Refaça a contagem de valores *missings*.  

4. Apgar é uma *nota* que o pediatra dá ao bebê quando nasce de acordo com algumas características associadas principalmente à respiração. Apgar 1 e Apgar 5 são as notas 1 e 5 minutos do nascimento. Apgar5 será a nossa variável de interesse principal. Então remova todos os registros com Apgar5 não preenchido. Para esta seleção, conte novamente o número de linhas e o número de *missings*.  

5. observe que as variáveis ```['ESTCIVMAE', 'CONSULTAS']``` possuem o código ```9```, que significa *ignorado*. Vamos assumir que o não preenchido é o mesmo que o código ```9```.<br>
6. Substitua os valores faltantes da quantitativa (```QTDFILVIVO```) por zero.  
7. Das restantes, decida que valore te parece mais adequado (um 'não preenchido' ou um valor 'mais provável' como no item anterior) e preencha. Justifique. Lembre-se de que tratamento de dados é trabalho do cientista, e que estamos tomando decisões a todo o momento - não há necessariamente certo e errado aqui.  
8. O Apgar possui uma classificação indicando se o bebê passou por asfixia:
- Entre 8 e 10 está em uma faixa 'normal'.
- Entre 6 e 7, significa que o recém-nascido passou por 'asfixia leve'.
- Entre 4 e 5 significa 'asfixia moderada'.
- Entre 0 e 3 significa 'asfixia severa'.  

Crie uma categorização dessa variável com essa codificação e calcule as frequências dessa categorização.  
<br>
9. Renomeie as variáveis para que fiquem no *snake case*, ou seja, em letras minúsculas, com um *underscore* entre as palávras. Dica: repare que se você não quiser criar um *dataframe* novo, você vai precisar usar a opção ```inplace = True```.

In [21]:
import pandas as pd
import requests

# 1) seu código aqui
sinasc = pd.read_csv('SINASC_RO_2019.csv')
print(sinasc.shape)
sinasc.drop_duplicates().shape
# Não há duplicados

(27028, 69)


(27028, 69)

In [22]:
# 2) Conte o número de valores missing por variável.
sinasc.isna().sum().sort_values(ascending=False)

DTRECORIGA    27028
CODANOMAL     26814
IDADEPAI      19421
SERIESCMAE    12710
DTULTMENST    10072
              ...  
PESO              0
DTCADASTRO        0
NUMEROLOTE        0
VERSAOSIST        0
ORIGEM            0
Length: 69, dtype: int64

In [23]:
# 3) Então crie uma seleção dessa base somente com as colunas que interessam.
df = sinasc.loc[:,['LOCNASC', 'IDADEMAE', 'ESTCIVMAE', 'ESCMAE', 'QTDFILVIVO', 'GESTACAO', 'GRAVIDEZ', 'CONSULTAS', 'APGAR5']]
df.shape

(27028, 9)

In [24]:
# 4) Então remova todos os registros com Apgar5 não preenchido. Para esta seleção, conte novamente o número de linhas e o número de missings.
df.dropna(subset='APGAR5', inplace=True)
df.shape

(26925, 9)

In [25]:
# 5) observe que as variáveis ['ESTCIVMAE', 'CONSULTAS'] possuem o código 9, que significa ignorado. Vamos assumir que o não preenchido é o mesmo que o código 9
df['ESTCIVMAE'].fillna(9, inplace=True)
df['CONSULTAS'].fillna(9, inplace=True)
df.loc[:,['ESTCIVMAE','CONSULTAS']].isna().sum()

ESTCIVMAE    0
CONSULTAS    0
dtype: int64

In [26]:
# 6) Substitua os valores faltantes da quantitativa (QTDFILVIVO) por zero.
df['QTDFILVIVO'].fillna(0, inplace=True)
df['QTDFILVIVO'].isna().sum()

0

In [27]:
# 7) Das restantes, decida que valore te parece mais adequado (um 'não preenchido' ou um valor 'mais provável' como no item anterior) e preencha. Justifique

#vendo quais colunas têm dados faltantes
print(df.isna().sum().sort_values(ascending=False))

# vendo valores existentes para GESTACAO, ESCMAE, GRAVIDEZ
print('\nGESTACAO')
print(df['GESTACAO'].value_counts())

print('\nESCMAE')
print(df['ESCMAE'].value_counts())

print('\nGRAVIDEZ')
print(df['GRAVIDEZ'].value_counts())

#não identifiquei um valor pré-existente para substituir os faltantes, logo vou preencher com 'nao preenchido'd
df[['GESTACAO','ESCMAE','GRAVIDEZ']] = df[['GESTACAO','ESCMAE','GRAVIDEZ']].fillna('Nao preenchido')

#vendo se dados faltantes
print(f'\n{df.isna().sum().sort_values(ascending=False)}')

GESTACAO      1216
ESCMAE         310
GRAVIDEZ        76
LOCNASC          0
IDADEMAE         0
ESTCIVMAE        0
QTDFILVIVO       0
CONSULTAS        0
APGAR5           0
dtype: int64

GESTACAO
37 a 41 semanas        22523
32 a 36 semanas         2234
42 semanas e mais        698
28 a 31 semanas          162
22 a 27 semanas           85
Menos de 22 semanas        7
Name: GESTACAO, dtype: int64

ESCMAE
8 a 11 anos        15581
12 anos ou mais     5757
4 a 7 anos          4727
1 a 3 anos           510
Nenhuma               40
Name: ESCMAE, dtype: int64

GRAVIDEZ
Única              26356
Dupla                488
Tríplice e mais        5
Name: GRAVIDEZ, dtype: int64

LOCNASC       0
IDADEMAE      0
ESTCIVMAE     0
ESCMAE        0
QTDFILVIVO    0
GESTACAO      0
GRAVIDEZ      0
CONSULTAS     0
APGAR5        0
dtype: int64


In [31]:
# 8) Crie uma categorização dessa variável com essa codificação e calcule as frequências dessa categorização.

df.loc[(df['APGAR5'] >=8.0) & (df['APGAR5'] <=10.0), 'CLASSE_APGAR'] = 'normal'
df.loc[(df['APGAR5'] >=6.0) & (df['APGAR5'] <=7.0), 'CLASSE_APGAR'] = 'asfixia leve'
df.loc[(df['APGAR5'] >=4.0) & (df['APGAR5'] <=5.0), 'CLASSE_APGAR'] = 'asfixia moderada'
df.loc[(df['APGAR5'] >=0.0) & (df['APGAR5'] <=3.0), 'CLASSE_APGAR'] = 'asfixia severa'

print(f"\n{df['CLASSE_APGAR'].value_counts()}")


normal              26463
asfixia leve          320
asfixia severa         74
asfixia moderada       68
Name: CLASSE_APGAR, dtype: int64


In [32]:
# 9) Renomeie as variáveis para que fiquem no snake case

df.columns = ['loc_nasc', 'idade_mae', 'est_civ_mae', 'esc_mae', 'qtd_fil_vivo', 'gestacao', 'gravidez', 'consultas', 'apgar5', 'classe_apgar']
df.head()

,loc_nasc,idade_mae,est_civ_mae,esc_mae,qtd_fil_vivo,gestacao,gravidez,consultas,apgar5,classe_apgar
0,1,19,5.0,8 a 11 anos,0.0,37 a 41 semanas,Única,4,10.0,normal
1,1,29,2.0,8 a 11 anos,1.0,37 a 41 semanas,Única,4,9.0,normal
2,1,37,9.0,8 a 11 anos,2.0,37 a 41 semanas,Única,4,10.0,normal
3,1,30,5.0,12 anos ou mais,0.0,37 a 41 semanas,Única,3,10.0,normal
4,1,30,2.0,8 a 11 anos,1.0,37 a 41 semanas,Única,4,10.0,normal
